In [2]:
from collections import defaultdict, Counter
from intervaltree import Interval, IntervalTree
import itertools
import functools
import math
import numpy as np
import re
from typing import Callable, TypeVar

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

In [99]:
data1 = data(1)

def day1(match):
    sum = 0
    for line in data1:
        matched = match(line)
        first, last = matched[0], matched[-1]
        # print(first, last, sum)
        sum += int(first + last)
    return sum

print(day1(lambda x: re.findall(r'\d', x)))

def part2match(x):
    change = {
        'one': '1',
        'two': '2',
        'three': '3',
        'four': '4',
        'five': '5',
        'six': '6',
        'seven': '7',
        'eight': '8',
        'nine': '9',
    }
    matched = re.findall(r'(?=(\d|one|two|three|four|five|six|seven|eight|nine))', x)
    return [change[match] if match in change else match for match in matched]

day1(part2match)
    

55816


54980

In [148]:
def format2(line):
    def tuplefy(round):
        result = [0,0,0]
        for i in round:
            result[0 if 'blue' in i else 1 if 'green' in i else 2] += int(i.split(' ')[0])
        return result

    line = line.split(': ')[-1]
    rounds = line.split('; ' )
    return np.array([tuplefy(x.split(', ')) for x in rounds])

data2 = data(2, format2)

def day2_1(start):
    result = 0
    for i, game in enumerate(data2):
        possible = np.subtract(start, game)
        if np.any(possible < 0):
            continue
        result += i+1
    return result

print(day2_1((14, 13, 12)))

def day2_2():
    result = 0
    for game in data2:
        requirements = np.amax(game, axis=0)
        result += functools.reduce(lambda x, y: x*y, requirements, 1)
    return result
day2_2()

2377


71220

In [130]:
data3 = data(3)

def day3():
    numbers = []
    for i, line in enumerate(data3):
        numbers += [(m.group(), (i, m.start())) for m in re.finditer(r'\d+', line)]

    array3 = np.pad(np.array([list(line) for line in data3]), ((1, 1), (1, 1)), constant_values='.')
    gears = defaultdict(lambda: [])

    part1 = 0
    for n, (i, j) in numbers:
        sliced = array3[i:i+3, j:j+2+len(n)]
        stringified =  ''.join(''.join(x) for x in sliced)
        if re.search(r'[^\d.]', stringified):
            part1 += int(n)
        if stars := np.where(sliced == '*'):
            for x, _ in enumerate(stars[0]):
                gears[(i+stars[0][x], j+stars[1][x])].append(int(n))

    part2 = 0
    for g in gears:
        if len(gears[g]) == 2:
            part2 += gears[g][0] * gears[g][1]

    return part1, part2

day3()

(521515, 69527306)

In [5]:
def parse4(line):
    winning, have = map(
        lambda numbers: {int(x) for x in numbers.split()},
        line.split(':')[-1].split('|')
    )
    return winning, have

data4 = data(4, parse4)

def day4(data):
    result, cards = 0, Counter()
    for i, line in enumerate(data):
        matches = line[0].intersection(line[1])
        if matches:
            result += 2**(len(matches)-1)
        for n in range(len(matches)):
            cards[i+1+n] += cards[i]+1
    return result, cards.total() + len(data)

day4(data4)


(25651, 19499881)

In [39]:
def day5_maps(data):
    names, maps = [], []

    def create_map(lines):
        ranges = IntervalTree([Interval(0, math.inf)])
        name = lines[0].split()[0]
        for i, line in enumerate(lines[1:]):
            if not line:
                break
            dest, source, size = map(int, line.split())
            ranges.chop(source, source+size)
            ranges[source:source+size] = dest
        return name, ranges, lines[i+2:]

    seeds, data = map(int, data[0].split(': ')[-1].split()), data[2:]

    while len(data):
        name, intervals, data = create_map(data)
        names.append(name)
        maps.append(intervals)

    return list(seeds), maps, names

seeds, intervals, names = day5_maps(data(5))

def day5_1(seeds, data):
    lowest = math.inf
    for seed in seeds:
        for interval in data:
            start, end, dest = next(iter(interval.at(seed)))
            seed = seed if dest is None else (seed-start)+dest
        lowest = min(lowest, seed)
    return lowest

print(day5_1(seeds, intervals))


1181555926


In [38]:

def day5_2(seeds, data):
    seedrange = IntervalTree([Interval(start, start+size) for start, size in zip(seeds[::2], seeds[1::2])])
    seedrange.merge_overlaps()

    for i, interval in enumerate(data):
        newrange = IntervalTree()
        print(names[i], len(seedrange), seedrange)
        for seed in seedrange:
            seedstart, seedend = seed[0], seed[1]
            intersections = interval[seedstart:seedend]
            # print('intersections', seed, len(intersections), intersections)
            for hit in intersections:
                start, end, dest = hit
                newstart, newend = seed[0], seed[1]
                if dest is not None:
                    newstart = dest + max(seedstart, start)-start
                    newend = dest + min(seedend, end)-start
                newrange.addi(newstart, newend)
        newrange.merge_overlaps()
        seedrange = newrange
        
    # print(seedrange)
    lowest = min([loc[0] for loc in seedrange])
    return lowest

day5_2(seeds, intervals)
# 817855325
# 1174966665

seed-to-soil 2 IntervalTree([Interval(55, 68), Interval(79, 93)])
soil-to-fertilizer 2 IntervalTree([Interval(57, 70), Interval(81, 95)])
fertilizer-to-water 2 IntervalTree([Interval(57, 70), Interval(81, 95)])
water-to-light 3 IntervalTree([Interval(53, 57), Interval(57, 70), Interval(81, 95)])
light-to-temperature 3 IntervalTree([Interval(46, 50), Interval(50, 63), Interval(74, 88)])
temperature-to-humidity 4 IntervalTree([Interval(45, 56), Interval(78, 81), Interval(82, 86), Interval(86, 99)])
humidity-to-location 4 IntervalTree([Interval(46, 57), Interval(78, 81), Interval(82, 86), Interval(86, 99)])


46

In [23]:
def day5_2(seeds, data):
    lowest = math.inf

    for i in range(len(seeds)//2):
        seedrange = IntervalTree()
        seedrange.addi(seeds[i*2], seeds[i*2]+seeds[i*2+1])
    # Indented here - unclear why this is a different result
        for interval in data:
            for seed in seedrange:
                seedstart, seedend = seed[0], seed[1]
                intersections = interval[seedstart:seedend]
                # print(intersections, seed)
                newrange = IntervalTree()
                for hit in intersections:
                    start, end, dest = hit
                    if dest is not None:
                        newstart = dest + max(seedstart,start)-start
                        newend = dest + min(seedend, end)-start
                    # print('hit', start, end, seedstart, seedend, newstart,newend)
                    newrange.addi(newstart, newend)
                print('a', seedrange, 'new', newrange)
            newrange.merge_overlaps()
            seedrange = newrange
            
        # print(seedrange)
        for loc in seedrange:
            # print(loc[0], lowest)
            lowest = min(loc[0], lowest)
    return lowest

day5_2(*data5)

a IntervalTree([Interval(364807853, 773420016)]) new IntervalTree([Interval(0, 18343754), Interval(18343754, 77244511), Interval(335002083, 391285622), Interval(498335903, 586838162), Interval(2070957389, 2227672101), Interval(3100147259, 3111376183)])
a IntervalTree([Interval(0, 18343754), Interval(18343754, 77244511), Interval(335002083, 391285622), Interval(498335903, 586838162), Interval(2070957389, 2227672101), Interval(3100147259, 3111376183)]) new IntervalTree([Interval(1707369117, 1725712871)])
a IntervalTree([Interval(0, 18343754), Interval(18343754, 77244511), Interval(335002083, 391285622), Interval(498335903, 586838162), Interval(2070957389, 2227672101), Interval(3100147259, 3111376183)]) new IntervalTree([Interval(990797653, 1079299912)])
a IntervalTree([Interval(0, 18343754), Interval(18343754, 77244511), Interval(335002083, 391285622), Interval(498335903, 586838162), Interval(2070957389, 2227672101), Interval(3100147259, 3111376183)]) new IntervalTree([Interval(210693872

817855325